In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from astropy.table import Table
import astropy.units as u
import astropy.constants as c
import astropy.coordinates as coords

In [ ]:
plt.rc('figure', dpi=120)

In [ ]:
10.*u.earthMass/(0.02*u.solMass).to(u.earthMass)

In [ ]:
from destroyer import RockDerivative, GradientSpectra, Spectra

In [ ]:
rd = RockDerivative()
dXHdm = rd.dXHdm_avg(0, 20)

In [ ]:
g = GradientSpectra()

In [ ]:
res = np.array([g.gradient[z]*dXHdm[i] for i, z in rd.df.Z[2:].iteritems()])
netres = res.sum(axis=0)

In [ ]:
d = Spectra()

In [ ]:
from numpy import ma

In [ ]:
maskedspec = ma.array(d.spec, mask=d.mask, fill_value=0)
maskedivar = ma.array(d.ivar, mask=d.mask, fill_value=1)

In [ ]:
print("number of wavelength pixels = {:d}".format(g.wave.size))

In [ ]:
from matplotlib import ticker

In [ ]:
plt.hist(d.mask.sum(axis=1), np.logspace(0,4,64), log=True);
plt.xscale('log');
plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter());
plt.axvline(g.wave.size, c='k', lw=.5,);
plt.axvline(g.wave.size*0.1, c='k', lw=.5, ls='dashed');
plt.axvline(g.wave.size*0.01, c='k', lw=.5, ls='dashed');
plt.xlabel("Number of masked pixels");
plt.ylabel("Count");
plt.tight_layout();
plt.savefig("plots/number_of_masked_pixels.png");

In [ ]:
randidx = np.random.randint(0, high=d.spec.shape[0], size=50)
plt.pcolor(g.wave, np.arange(50), d.mask[randidx], cmap='gray_r');
plt.xlabel(r"Wavelength [$\AA$]")
plt.title("Masks of random 50 star spectra");
plt.tight_layout()
plt.savefig("plots/random_50_masks.png");

In [ ]:
medspec = ma.median(maskedspec, axis=0)
diffspec = maskedspec - medspec

In [ ]:
plt.figure(figsize=(10,4));
plt.xlim(3800,9000)
plt.plot(g.wave, medspec, lw=.5);
plt.xlabel("Wavelength [$\AA$]")
plt.title("Median spectrum");
plt.tight_layout()
plt.savefig("plots/median_spec.png");

In [ ]:
beta = np.einsum('i,ji,ji->j', netres, diffspec.filled(), maskedivar.filled()) / np.einsum('i,ji->j', netres**2, maskedivar.filled())
model = np.einsum('i,j->ij', beta, netres)
chisq = ma.sum((diffspec-model)**2*maskedivar, axis=1)

In [ ]:
np.percentile(beta, [0,.01,1,15,50,75,99,99.9,100])

In [ ]:
plt.plot(beta, chisq, 'k,')
plt.xlim(-50,50);
plt.ylim(100, 1e5);
plt.yscale('log')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8,4))
ax[0].hist(beta, np.linspace(-50,50,101),);
# plt.axvspan(15,25,alpha=.5, color='gray');
ax[0].axvline(0, c='k', lw=1)
ax[0].set_xlabel(r"$\hat\beta^i$ [$M_\oplus$]")
ax[1].hist(beta, np.linspace(-100,100,101), log=True);
ax[1].set_xlabel(r"$\hat\beta^i$ [$M_\oplus$]");
fig.tight_layout()
for a in ax:
    plt.sca(a)
    plt.axvspan(-48,-55, alpha=.5, facecolor='tab:gray', );
    plt.axvspan(15,25, alpha=.5, facecolor='tab:red', );
fig.savefig("plots/hist_beta.png");

In [ ]:
percspec = np.nanpercentile(d.spec, [0,0.1,1,2,10,50,90,98,99,99.9,100], axis=0)

In [ ]:
outlier = (d.spec<percspec[1][None,:]) | (d.spec>percspec[-1][None,:])

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(percspec[[2,-2]].T, lw=.5);
# plt.plot(d.spec[0]-percspec[2], lw=.5, c='k')
# plt.plot(outlier[0], c='r')
# plt.axhline(0)
# plt.ylim(-0.1,.6)

In [ ]:
# wth = np.where((beta<-48) & (beta>-55))[0]
wth = np.where((beta>50) & (beta<100))[0]
wth

In [ ]:
chisq[wth]/(~d.mask)[wth].sum(axis=1)

In [ ]:
!mkdir plots/beta_50_100

In [ ]:
for j in wth:
    plt.figure(figsize=(10,4))
    plt.plot(g.wave, diffspec[j].T, lw=.5);
    # smoothed_signal = convolve(diffspec[j], Box1DKernel(5))
    # plt.plot(g.wave, smoothed_signal, lw=1)
    plt.plot(g.wave, model[j], lw=.5)
    plt.title(r"{:d} $\beta$={:.2f}".format(j,beta[j]));
    plt.tight_layout()
    plt.savefig("plots/beta_50_100/{:d}.png".format(j))
    plt.close()

In [ ]:
from astropy.convolution import convolve, Box1DKernel

In [ ]:
smoothed_signal = convolve(diffspec[j], Box1DKernel(5))

In [ ]:
plt.figure(figsize=(10,4))
# plt.plot(wave, diffspec[j], lw=1)
plt.plot(wave, smoothed_signal)
plt.plot(wave, netres*s[j], lw=1)
# plt.plot(wave, diffspec[j] - netres*s[j] - 0.2, lw=.5)
# plt.xlim(3800,5000)
# plt.xlim(5000,6000)
# plt.xlim(8000,9000)
plt.ylim(-.2,.2)